In [1]:
import random

class MerkleHellmanCryptosystem:
    """
    Implements the Merkle-Hellman knapsack cryptosystem.
    
    Attributes:
        length (int): The length of the superincreasing sequence, public key, and messages.
        public_key (list of int): The public key used for encryption.
        private_key (tuple): The private key used for decryption, containing the superincreasing sequence, modulus q, and multiplier w.
    """

    def __init__(self, length):
        """
        Initializes the cryptosystem with a specific length for keys and messages.
        
        Parameters:
            length (int): The length of the keys and messages.
        """
        self.length = length
        self.public_key, self.private_key = self.generate_keys(length)

    def generate_keys(self, n):
        """
        Generates the public and private keys for the cryptosystem.
        
        Parameters:
            n (int): The length of the superincreasing sequence.
        
        Returns:
            tuple: A tuple containing the public key (list of int) and private key (tuple of list, int, int).
        """
        # Initial superincreasing sequence for private key
        a_n = [random.randint(1, 100)]
        for i in range(1, n):
            a_n.append(sum(a_n) + random.randint(1, 10))
            
        # Modulus q, larger than the sum of the superincreasing sequence
        q = sum(a_n) + random.randint(1, 10)
        
        # Multiplier w that is coprime with q
        w = random_prime(q-1, True, q//2)  # random_prime available thanks to SageMath
        
        # Public key derived from private key elements
        public_key = [(w*a_i) % q for a_i in a_n]
        private_key = (a_n, q, w)
        
        return public_key, private_key

    def encrypt(self, message):
        """
        Encrypts a binary message using the public key.
        
        Parameters:
            message (list of int): The binary message to be encrypted.
        
        Returns:
            int: The encrypted message as an integer.
        """
        assert len(message) == len(self.public_key), "Message length must match the public key length."
        return sum(m_i * b_i for m_i, b_i in zip(message, self.public_key))

    def decrypt(self, cipher):
        """
        Decrypts an encrypted message using the private key.
        
        Parameters:
            cipher (int): The encrypted message.
        
        Returns:
            list of int: The decrypted binary message.
        """
        a_n, q, w = self.private_key
        w_inv = power_mod(w, -1, q)  # power_mod available thanks to SageMath
        c_prim = (cipher * w_inv) % q

        message = []
        for a_i in reversed(a_n):
            if a_i <= c_prim:
                message.append(1)
                c_prim -= a_i
            else:
                message.append(0)
        message.reverse()
        return message

    @staticmethod
    def generate_message(length):
        """
        Generates a random binary message of a specified length.
        
        Parameters:
            length (int): The length of the message to generate.
        
        Returns:
            list of int: A random binary message.
        """
        return [random.randint(0, 1) for _ in range(length)]

    @staticmethod
    def error_count(arr1, arr2):
        """
        Counts the number of differences between two arrays.
        
        Parameters:
            arr1 (list of int): The first array.
            arr2 (list of int): The second array.
        
        Returns:
            int: The number of differences between the two arrays.
        
        Raises:
            ValueError: If the arrays have different lengths.
        """
        if len(arr1) != len(arr2):
            raise ValueError("Both arrays must have the exact same length!")
        return sum(1 for x, y in zip(arr1, arr2) if x != y)

    def print_info(self, original_message, encrypted_message, decrypted_message):
        """
        Prints information about the encryption and decryption process.
        
        Parameters:
            original_message (list of int): The original binary message.
            encrypted_message (int): The encrypted message.
            decrypted_message (list of int): The decrypted binary message.
        """
        print("Length:", self.length)
        print("Public key:", self.public_key)
        print("Private key:", self.private_key)
        print("Original message:", original_message)
        print("Encrypted message:", encrypted_message)
        print("Decrypted message:", decrypted_message)
        print("Errors:", self.error_count(original_message, decrypted_message))

    def run(self, message=None):
        """
        Demonstrates the complete encryption and decryption process using randomly generated messages.
        
        Parameters:
            message (list of int): Optional parameter representing a binary message. If it is missing, a random message is generated.
        """
        if message is None:
            original_message = self.generate_message(self.length)
        else:
            assert len(message) == self.length, "Provided message length must match the cryptosystem length."
            original_message = message
        encrypted_message = self.encrypt(original_message)
        decrypted_message = self.decrypt(encrypted_message)
        self.print_info(original_message, encrypted_message, decrypted_message)


In [2]:

length = ZZ.random_element(1, 100)  
mh_cryptosystem = MerkleHellmanCryptosystem(length)
mh_cryptosystem.run()


Length: 32
Public key: [125637372816, 26746345198, 124159303377, 121414317276, 73482146730, 34277787689, 26113377034, 24002339431, 62222462569, 82213879571, 107978929868, 18598110054, 121658311242, 31739089095, 63689330967, 474372456, 948744912, 44128535391, 88257070782, 7167653742, 85001920465, 99548380726, 72192471974, 116160529311, 62974570800, 69500312326, 40320749811, 80641499622, 6154295129, 12308590258, 52841595153, 91676559376]
Private key: ([54, 62, 117, 234, 474, 943, 1892, 3785, 7565, 15127, 30256, 60510, 121026, 242055, 484101, 968202, 1936404, 3872811, 7745622, 15491250, 30982493, 61964984, 123929968, 247859937, 495719880, 991439762, 1982879523, 3965759046, 7931518093, 15863036186, 31726072371, 63452144738], 126904289478, 98679874841)
Original message: [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
Encrypted message: 880437410504
Decrypted message: [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 

In [3]:
length = 6
other_cryptosystem = MerkleHellmanCryptosystem(length)
fixed_message = [0,1,1,0,0,1]
other_cryptosystem.run(fixed_message)

Length: 6
Public key: [2502, 2462, 2027, 1102, 2204, 1471]
Private key: ([86, 94, 181, 366, 732, 1465], 2932, 2927)
Original message: [0, 1, 1, 0, 0, 1]
Encrypted message: 5960
Decrypted message: [0, 1, 1, 0, 0, 1]
Errors: 0
